In [1]:
import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import pandas as pd

In [5]:
from ipynb.fs.full.preprocessing import * 
from ipynb.fs.full.WordProcessing import my_dict 

['buy', 'nok', 'we', 'are', 'in', 'power']


In [9]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

filtered_dict = {}
for i in my_dict.keys():
    filtered_dict[i] = []
    for w in my_dict[i]:
        if w not in stop_words:
            filtered_dict[i].append(w)
print(filtered_dict[358])

['buy', 'nok', 'power']


In [ ]:
analyzer = SentimentAnalyzer()

In [ ]:
analyzer.save_file(analyzer, 'sentiment_analyzer')